In [1]:
import pandas as pd
import numpy as np
from sklearn import tree

In [2]:
#botdata = pd.read_csv('botAccounts.csv')
#gooddata= pd.read_csv('GoodAccounts.csv', encoding = 'ISO-8859-1')

In [2]:
testbot = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/bots_data.csv", encoding = 'ISO-8859-1')
testgood = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/nonbots_data.csv", encoding = 'ISO-8859-1')

In [3]:
df = testbot.append(testgood)

In [56]:
df.head()

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listedcount,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
0,3.982732e+09,3982731976,mcgucket_bot,,A bot that tweets every line said by Fiddlefor...,,1129,7,2,Sat Oct 17 22:35:31 +0000 2015,0,False,23557,en,"{'retweeted': False, 'is_quote_status': False,...",False,False,False,McGucket bot,1
1,8.410000e+17,8.41E+17,BowieK66,,,,0,22,0,13/03/2017 22:21,0,False,1,en,Status(_api=<tweepy.api.API object at 0x101927...,True,True,False,MJ Kuhn,1
2,2.768375e+09,2768374981,ducknoteprice,,,,3,0,3,Mon Aug 25 23:17:59 +0000 2014,0,False,1050,en,{'created_at': 'Fri Feb 26 17:01:15 +0000 2016...,True,False,False,duckNote,1
3,3.304189e+09,3304189373,robotrecipes,robot kitchen,tasty recipes for robot // not for human // a ...,http://t.co/PdagJGqVMR,505,13,49,Sat May 30 23:42:16 +0000 2015,0,False,5109,en,"{u'contributors': None, u'truncated': False, u...",False,False,False,robot recipes,1
4,3.224289e+09,3224289024,everyumlaut,,bot by @dbaker_h,,15,0,11,Sat May 23 14:39:21 +0000 2015,0,False,31365,en-gb,"{u'contributors': None, u'truncated': False, u...",True,False,False,ÌÇvÌÇrÌÀ Ì_mlÌ_Ì_¼Ñ,1


In [46]:
def adjust_column_name(df):
    for x in df.columns.values:
        df = df.rename(columns={x: x.lower()})
    df = df.rename(columns={'listed_count' : 'listedcount'})

In [6]:
from sklearn.cross_validation import train_test_split
train, test = train_test_split(df, test_size = 0.2)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB

In [67]:
def nb_dt(nb_column_name):
    df = testbot.append(testgood).fillna("")
    train, test = train_test_split(df, test_size = 0.2)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count','nb_result']
    count_vect = CountVectorizer()
    def convert(column):
        X_word_counts = count_vect.fit_transform(column)
        count_vect.vocabulary_.get(u'algorithm')
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    def convert2(column):
        X_word_counts = count_vect.transform(test[nb_column_name])
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    #Train
    clf = tree.DecisionTreeClassifier(random_state=0)
    M_clf = MultinomialNB().fit(convert(train[nb_column_name]), train['bot'])
    train['nb_result'] = M_clf.predict(convert(train[nb_column_name]))
    clf.fit(train[numfeatures],train["bot"])
    #Test
    test['nb_result'] = M_clf.predict(convert2(train[nb_column_name]))
    return np.mean(clf.predict(test[numfeatures])==test['bot'])

In [70]:
for x in ['screen_name', 'name', 'description', 'status']:
    print(nb_dt(x))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.498881431767
0.818791946309
0.870246085011
0.82774049217


In [44]:
def dt():
    df = testbot.append(testgood)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count']
    train, test = train_test_split(df, test_size = 0.2)
    clf.fit(train[numfeatures],train["bot"])
    return np.mean(clf.predict(test[numfeatures])==test['bot'])

In [45]:
sum(dt() for _ in range(20))/20.0

0.84250559284116322